# Importing Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pydicom as dicom
import os, sys, time, shutil, scipy, cv2, json, datetime
import PIL.Image
from tqdm.notebook import tqdm

import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split

%reload_ext autoreload
%autoreload 2
from utils import *


# Train-Validation-Test Split

In [3]:
df = pd.read_csv('../data/train.csv', index_col=0)
df.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0


In [8]:
df2 = pd.read_csv('../data/tr.csv', index_col=0)
df2.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target
0,ISIC_0065823,NaN,BCN_0003902,male,75.0,anterior torso,NaN,NaN,1
1,ISIC_0013908_downsampled,NaN,MSK4_0010423,male,75.0,posterior torso,NaN,NaN,1
2,ISIC_0054089,NaN,BCN_0005268,male,45.0,anterior torso,NaN,NaN,1
3,ISIC_0061371,NaN,BCN_0004405,male,60.0,palms/soles,NaN,NaN,1
4,ISIC_8702237,IP_0738123,IL_8854900,female,50.0,lower extremity,melanoma,malignant,1


In [9]:
df2.target.value_counts()

0    26033
1     4084
Name: target, dtype: int64

In [4]:
df.target.value_counts()

0    32542
1     5106
Name: target, dtype: int64

In [5]:
df.file.value_counts()

ISIC_9745942    1
ISIC_7432725    1
ISIC_0059816    1
ISIC_1158466    1
ISIC_4957778    1
               ..
ISIC_6705470    1
ISIC_0072966    1
ISIC_2354133    1
ISIC_9094080    1
ISIC_0928982    1
Name: file, Length: 37648, dtype: int64

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37648 entries, 0 to 37647
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   file        37648 non-null  object 
 1   patient_id  33126 non-null  object 
 2   lesion_id   37311 non-null  object 
 3   gender      37502 non-null  object 
 4   age         37495 non-null  float64
 5   site        36994 non-null  object 
 6   diagnosis   33126 non-null  object 
 7   ben_mal     33126 non-null  object 
 8   target      37648 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 2.9+ MB


In [10]:
melanoma = df[df.target == 1]
non_melanoma = df[df.target == 0]
y_mel = melanoma.target
X_mel = melanoma.drop(columns=['target'], axis=1)
y_non_mel = non_melanoma.target
X_non_mel = non_melanoma.drop(columns=['target'], axis=1)

In [11]:
X_tr_m, X_val_m, y_tr_m, y_val_m = train_test_split(X_mel, y_mel, random_state=42, test_size=0.2)
X_tr_nm, X_val_nm, y_tr_nm, y_val_nm = train_test_split(X_non_mel, y_non_mel, random_state=42, test_size=0.2)

mel_train = pd.concat([X_tr_m, y_tr_m], axis=1)
non_mel_train = pd.concat([X_tr_nm, y_tr_nm], axis=1)

train = pd.concat([mel_train, non_mel_train], axis=0)
train = train.reset_index(drop=True)

In [12]:
X_val_m2, X_tt_m, y_val_m2, y_tt_m = train_test_split(X_val_m, y_val_m, random_state=42, test_size=0.5)
X_val_nm2, X_tt_nm, y_val_nm2, y_tt_nm = train_test_split(X_val_nm, y_val_nm, random_state=42, test_size=0.5)

mel_val = pd.concat([X_val_m2, y_val_m2], axis=1)
non_mel_val = pd.concat([X_val_nm2, y_val_nm2], axis=1)
mel_tt = pd.concat([X_tt_m, y_tt_m], axis=1)
non_mel_tt = pd.concat([X_tt_nm, y_tt_nm], axis=1)

validation = pd.concat([mel_val, non_mel_val], axis=0)
validation = validation.reset_index(drop=True)

test = pd.concat([mel_tt, non_mel_tt], axis=0)
test = test.reset_index(drop=True)

In [13]:
train.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target
0,ISIC_0065823,NaN,BCN_0003902,male,75.0,anterior torso,NaN,NaN,1
1,ISIC_0013908_downsampled,NaN,MSK4_0010423,male,75.0,posterior torso,NaN,NaN,1
2,ISIC_0054089,NaN,BCN_0005268,male,45.0,anterior torso,NaN,NaN,1
3,ISIC_0061371,NaN,BCN_0004405,male,60.0,palms/soles,NaN,NaN,1
4,ISIC_8702237,IP_0738123,IL_8854900,female,50.0,lower extremity,melanoma,malignant,1


In [14]:
len(train)

30117

In [57]:
train.target.value_counts()

0    26033
1     4084
Name: target, dtype: int64

In [15]:
validation.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target
0,ISIC_0066678,NaN,BCN_0003144,male,45.0,anterior torso,NaN,NaN,1
1,ISIC_0057668,NaN,BCN_0003907,female,65.0,anterior torso,NaN,NaN,1
2,ISIC_0053843,NaN,BCN_0002433,male,45.0,anterior torso,NaN,NaN,1
3,ISIC_0072218,NaN,BCN_0003200,female,75.0,upper extremity,NaN,NaN,1
4,ISIC_0032685,NaN,HAM_0005608,male,75.0,posterior torso,NaN,NaN,1


In [16]:
len(validation)

3765

In [17]:
validation.target.value_counts()

0    3254
1     511
Name: target, dtype: int64

In [18]:
test.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target
0,ISIC_0064868,NaN,BCN_0003025,male,50.0,anterior torso,NaN,NaN,1
1,ISIC_0032617,NaN,HAM_0006101,male,75.0,posterior torso,NaN,NaN,1
2,ISIC_0061378,NaN,BCN_0001755,male,60.0,anterior torso,NaN,NaN,1
3,ISIC_0065382,NaN,BCN_0001728,male,50.0,anterior torso,NaN,NaN,1
4,ISIC_5732201,IP_8142871,IL_8516895,female,45.0,torso,melanoma,malignant,1


In [19]:
len(test)

3766

In [63]:
test.target.value_counts()

0    3255
1     511
Name: target, dtype: int64

In [64]:
train.to_csv("../data/tr.csv")
validation.to_csv("../data/val.csv")
test.to_csv("../data/tt.csv")

# Creating Training Dataset

In [24]:
train['jpg'] = train.file.apply(lambda x: str(x) + '.jpg')
mel_tr = train[train.target == 1]
nm_tr = train[train.target == 0]

mel_tr_ids = mel_tr.jpg.to_list()
nm_tr_ids = nm_tr.jpg.to_list()

In [46]:
images_dir = '../jpegs'
tr_mel = '../split/train/mel'
tr_nm = '../split/train/not_mel'

In [47]:
mel1 = mel_tr_ids[0:1000]
mel2 = mel_tr_ids[1000:2000]
mel3 = mel_tr_ids[2000:3000]
mel4 = mel_tr_ids[3000:]

In [48]:
for img in tqdm(mel1):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_mel, img))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [49]:
for img in tqdm(mel2):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_mel, img))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [50]:
for img in tqdm(mel3):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_mel, img))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [51]:
for img in tqdm(mel4):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_mel, img))

  0%|          | 0/1084 [00:00<?, ?it/s]

In [52]:
nm1 = nm_tr_ids[0:2000]
nm2 = nm_tr_ids[2000:4000]
nm3 = nm_tr_ids[4000:6000]
nm4 = nm_tr_ids[6000:8000]
nm5 = nm_tr_ids[8000:10000]
nm6 = nm_tr_ids[10000:12000]
nm7 = nm_tr_ids[12000:14000]
nm8 = nm_tr_ids[14000:16000]
nm9 = nm_tr_ids[16000:18000]
nm10 = nm_tr_ids[18000:20000]
nm11 = nm_tr_ids[20000:22000]
nm12 = nm_tr_ids[22000:24000]
nm13 = nm_tr_ids[24000:]

In [62]:
for img in tqdm(nm1):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [63]:
for img in tqdm(nm2):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [64]:
for img in tqdm(nm3):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [65]:
for img in tqdm(nm4):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [66]:
for img in tqdm(nm5):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [67]:
for img in tqdm(nm6):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [68]:
for img in tqdm(nm7):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [69]:
for img in tqdm(nm8):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [84]:
for img in tqdm(nm9):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [85]:
for img in tqdm(nm10):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [87]:
for img in tqdm(nm11):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [88]:
for img in tqdm(nm12):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [89]:
for img in tqdm(nm13):
    shutil.move(os.path.join(images_dir, img), os.path.join(tr_nm, img))

  0%|          | 0/2033 [00:00<?, ?it/s]

In [23]:
train = pd.read_csv('../data/tr.csv')
train

,Unnamed: 0,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target
0,0,ISIC_0065823,NaN,BCN_0003902,male,75.0,anterior torso,NaN,NaN,1
1,1,ISIC_0013908_downsampled,NaN,MSK4_0010423,male,75.0,posterior torso,NaN,NaN,1
2,2,ISIC_0054089,NaN,BCN_0005268,male,45.0,anterior torso,NaN,NaN,1
3,3,ISIC_0061371,NaN,BCN_0004405,male,60.0,palms/soles,NaN,NaN,1
4,4,ISIC_8702237,IP_0738123,IL_8854900,female,50.0,lower extremity,melanoma,malignant,1
...,...,...,...,...,...,...,...,...,...,...
30112,30112,ISIC_9166979,IP_7279968,IL_5062492,male,45.0,torso,unknown,benign,0
30113,30113,ISIC_1738297,IP_9965542,IL_7933843,male,60.0,head/neck,unknown,benign,0
30114,30114,ISIC_0365032,IP_3908059,IL_2756378,female,60.0,upper extremity,unknown,benign,0
30115,30115,ISIC_4919947,IP_8663649,IL_9363786,male,45.0,lower extremity,unknown,benign,0


In [25]:
train.head()

,Unnamed: 0,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,0,ISIC_0065823,NaN,BCN_0003902,male,75.0,anterior torso,NaN,NaN,1,ISIC_0065823.jpg
1,1,ISIC_0013908_downsampled,NaN,MSK4_0010423,male,75.0,posterior torso,NaN,NaN,1,ISIC_0013908_downsampled.jpg
2,2,ISIC_0054089,NaN,BCN_0005268,male,45.0,anterior torso,NaN,NaN,1,ISIC_0054089.jpg
3,3,ISIC_0061371,NaN,BCN_0004405,male,60.0,palms/soles,NaN,NaN,1,ISIC_0061371.jpg
4,4,ISIC_8702237,IP_0738123,IL_8854900,female,50.0,lower extremity,melanoma,malignant,1,ISIC_8702237.jpg


In [21]:
train.target.value_counts()

0    26033
1     4084
Name: target, dtype: int64

## Creating Sample Training Set

In [27]:
strat_tr = stratified_sample(df = train, strata = ['target'], size = 0.5, seed = 42)
strat_tr.head()

,index,Unnamed: 0,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,22645,22645,ISIC_5247383,IP_2117218,IL_8859164,male,35.0,lower extremity,unknown,benign,0,ISIC_5247383.jpg
1,19797,19797,ISIC_4422476,IP_6175417,IL_9497108,female,45.0,upper extremity,unknown,benign,0,ISIC_4422476.jpg
2,18060,18060,ISIC_8124459,IP_9616517,IL_9968409,female,40.0,upper extremity,unknown,benign,0,ISIC_8124459.jpg
3,29506,29506,ISIC_1669883,IP_0301948,IL_6017615,male,45.0,upper extremity,nevus,benign,0,ISIC_1669883.jpg
4,17825,17825,ISIC_5842418,IP_9558959,IL_4319660,male,45.0,upper extremity,unknown,benign,0,ISIC_5842418.jpg


In [28]:
strat_tr.target.value_counts()              

0    13016
1     2042
Name: target, dtype: int64

In [29]:
samp_mel_tr = strat_tr[strat_tr.target == 1]
samp_nm_tr = strat_tr[strat_tr.target == 0]

samp_mel_tr_ids = samp_mel_tr.jpg.to_list()
samp_nm_tr_ids = samp_nm_tr.jpg.to_list()

In [32]:
len(samp_nm_tr_ids)

13016

In [36]:
tr_mel_ori = '../jpegs/split/train/1_mel'
tr_mel_dest = '../jpegs/sample2/train/1_mel'
tr_nm_ori = '../jpegs/split/train/0_not_mel'
tr_nm_dest = '../jpegs/sample2/train/0_not_mel'

In [38]:
for img in tqdm(samp_mel_tr_ids):
    shutil.copyfile(os.path.join(tr_mel_ori, img), os.path.join(tr_mel_dest, img))

  0%|          | 0/2042 [00:00<?, ?it/s]

In [40]:
sam_tr_1 = samp_nm_tr_ids[0:2000]
sam_tr_2 = samp_nm_tr_ids[2000:4000]
sam_tr_3 = samp_nm_tr_ids[4000:6000] 
sam_tr_4 = samp_nm_tr_ids[6000:8000] 
sam_tr_5 = samp_nm_tr_ids[8000:10000] 
sam_tr_6 = samp_nm_tr_ids[10000:] 


In [41]:
for img in tqdm(sam_tr_1):
    shutil.copyfile(os.path.join(tr_nm_ori, img), os.path.join(tr_nm_dest, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [42]:
for img in tqdm(sam_tr_2):
    shutil.copyfile(os.path.join(tr_nm_ori, img), os.path.join(tr_nm_dest, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [43]:
for img in tqdm(sam_tr_3):
    shutil.copyfile(os.path.join(tr_nm_ori, img), os.path.join(tr_nm_dest, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [44]:
for img in tqdm(sam_tr_4):
    shutil.copyfile(os.path.join(tr_nm_ori, img), os.path.join(tr_nm_dest, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [45]:
for img in tqdm(sam_tr_5):
    shutil.copyfile(os.path.join(tr_nm_ori, img), os.path.join(tr_nm_dest, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [46]:
for img in tqdm(sam_tr_6):
    shutil.copyfile(os.path.join(tr_nm_ori, img), os.path.join(tr_nm_dest, img))

  0%|          | 0/3016 [00:00<?, ?it/s]

# Creating Validation Dataset

In [53]:
validation['jpg'] = validation.file.apply(lambda x: str(x) + '.jpg')
mel_val = validation[validation.target == 1]
nm_val = validation[validation.target == 0]

mel_val_ids = mel_val.jpg.to_list()
nm_val_ids = nm_val.jpg.to_list()

In [54]:
images_dir = '../jpegs'
val_mel = '../split/val/mel'
val_nm = '../split/val/not_mel'

In [55]:
for img in mel_val_ids:
    shutil.move(os.path.join(images_dir, img), os.path.join(val_mel, img))

In [60]:
val_nm1 = nm_val_ids[0:2000]
val_nm3 = nm_val_ids[2000:4000]
val_nm2 = nm_val_ids[4000:]


In [58]:
for img in tqdm(val_nm1):
    shutil.move(os.path.join(images_dir, img), os.path.join(val_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [59]:
for img in tqdm(val_nm2):
    shutil.move(os.path.join(images_dir, img), os.path.join(val_nm, img))

  0%|          | 0/2509 [00:00<?, ?it/s]

In [61]:
for img in tqdm(val_nm3):
    shutil.move(os.path.join(images_dir, img), os.path.join(val_nm, img))

  0%|          | 0/2000 [00:00<?, ?it/s]

## Sample Validation Set

In [55]:
validation.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,ISIC_0066678,NaN,BCN_0003144,male,45.0,anterior torso,NaN,NaN,1,ISIC_0066678.jpg
1,ISIC_0057668,NaN,BCN_0003907,female,65.0,anterior torso,NaN,NaN,1,ISIC_0057668.jpg
2,ISIC_0053843,NaN,BCN_0002433,male,45.0,anterior torso,NaN,NaN,1,ISIC_0053843.jpg
3,ISIC_0072218,NaN,BCN_0003200,female,75.0,upper extremity,NaN,NaN,1,ISIC_0072218.jpg
4,ISIC_0032685,NaN,HAM_0005608,male,75.0,posterior torso,NaN,NaN,1,ISIC_0032685.jpg


In [48]:
validation.target.value_counts()

0    3254
1     511
Name: target, dtype: int64

## Creating Sample Validation Set

In [56]:
strat_val = stratified_sample(df = validation, strata = ['target'], size = 0.5, seed = 42)

In [57]:
strat_val.target.value_counts()

0    1627
1     255
Name: target, dtype: int64

In [51]:
val_mel_ori = '../jpegs/split/val/1_mel'
val_mel_dest = '../jpegs/sample/val/1_mel'
val_nm_ori = '../jpegs/split/val/0_not_mel'
val_nm_dest = '../jpegs/sample/val/0_not_mel'

In [58]:
samp_mel_val = strat_val[strat_val.target == 1]
samp_nm_val = strat_val[strat_val.target == 0]

samp_mel_val_ids = samp_mel_val.jpg.to_list()
samp_nm_val_ids = samp_nm_val.jpg.to_list()

In [59]:
for img in tqdm(samp_mel_val_ids):
    shutil.copyfile(os.path.join(val_mel_ori, img), os.path.join(val_mel_dest, img))

  0%|          | 0/255 [00:00<?, ?it/s]

In [60]:
for img in tqdm(samp_nm_val_ids):
    shutil.copyfile(os.path.join(val_nm_ori, img), os.path.join(val_nm_dest, img))

  0%|          | 0/1627 [00:00<?, ?it/s]

# Creating Testing Set

In [61]:
test['jpg'] = test.file.apply(lambda x: str(x) + '.jpg')
mel_tt = test[test.target == 1]
nm_tt = test[test.target == 0]

mel_tt_ids = mel_tt.jpg.to_list()
nm_tt_ids = nm_tt.jpg.to_list()

In [78]:
# direc = '../split/te/'

# os.mkdir('../split/test/mel')
# os.mkdir('../split/test/non_mel')

direc = '../split/val/mel/'
direc2 = '../split/val/not_mel'
direc3 = '../split/test/mel'
direc4 = '../split/test/not_mel'

In [81]:
for img in tqdm(nm_tt_ids):
    shutil.move(os.path.join(direc2, img), os.path.join(direc4, img))

  0%|          | 0/3255 [00:00<?, ?it/s]

In [80]:
for img in tqdm(mel_tt_ids):
    shutil.move(os.path.join(direc, img), os.path.join(direc3, img))

  0%|          | 0/511 [00:00<?, ?it/s]

In [62]:
test.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,ISIC_0064868,NaN,BCN_0003025,male,50.0,anterior torso,NaN,NaN,1,ISIC_0064868.jpg
1,ISIC_0032617,NaN,HAM_0006101,male,75.0,posterior torso,NaN,NaN,1,ISIC_0032617.jpg
2,ISIC_0061378,NaN,BCN_0001755,male,60.0,anterior torso,NaN,NaN,1,ISIC_0061378.jpg
3,ISIC_0065382,NaN,BCN_0001728,male,50.0,anterior torso,NaN,NaN,1,ISIC_0065382.jpg
4,ISIC_5732201,IP_8142871,IL_8516895,female,45.0,torso,melanoma,malignant,1,ISIC_5732201.jpg


In [63]:
test.target.value_counts()

0    3255
1     511
Name: target, dtype: int64

## Creating Sample Testing Set

In [64]:
strat_tt = stratified_sample(df = test, strata = ['target'], size = 0.5, seed = 42)

In [91]:
strat_tt.target.value_counts()

0    651
1    102
Name: target, dtype: int64

In [98]:
tt_mel_ori = '../split/test/mel'
tt_mel_dest = '../sample/test/mel'
tt_nm_ori = '../split/test/not_mel'
tt_nm_dest = '../sample/test/not_mel'

In [99]:
samp_mel_tt = strat_tt[strat_tt.target == 1]
samp_nm_tt = strat_tt[strat_tt.target == 0]

samp_mel_tt_ids = samp_mel_tt.jpg.to_list()
samp_nm_tt_ids = samp_nm_tt.jpg.to_list()

In [102]:
for img in tqdm(samp_mel_tt_ids):
    shutil.copyfile(os.path.join(tt_mel_ori, img), os.path.join(tt_mel_dest, img))

  0%|          | 0/102 [00:00<?, ?it/s]

In [103]:
for img in tqdm(samp_nm_tt_ids):
    shutil.copyfile(os.path.join(tt_nm_ori, img), os.path.join(tt_nm_dest, img))

  0%|          | 0/651 [00:00<?, ?it/s]